In [39]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [40]:
stock = pd.read_csv('/content/Stock_Data.csv', encoding='ISO-8859-1')
stock.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [41]:
 train = stock[stock['Date'] < '20150101']
 test = stock[stock['Date'] > '20141231']

In [42]:
#Removing Punctuations
data = train.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)


#Renaming Column names with the indexes
list1 = [i for i in range(25)]
new_index = [str(i) for i in list1]
data.columns = new_index
data.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,A hindrance to operations extracts from the...,Scorecard,Hughes instant hit buoys Blues,Jack gets his skates on at ice cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar s debut double,Southgate strikes Leeds pay the penalty,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl s successor drawn into scandal,The difference between men and women,Sara Denver nurse turned solicitor,Diana s landmine crusade put Tories in a panic,Yeltsin s resignation caught opposition flat f...,Russian roulette,Sold out,Recovering a title
1,Scorecard,The best lake scene,Leader German sleaze inquiry,Cheerio boyo,The main recommendations,Has Cubie killed fees,Has Cubie killed fees,Has Cubie killed fees,Hopkins furious at Foster s lack of Hannibal...,Has Cubie killed fees,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man s extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn t know without the ...,Millennium bug fails to bite
2,Coventry caught on counter by Flo,United s rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,...,South Melbourne Australia,Necaxa Mexico,Real Madrid Spain,Raja Casablanca Morocco,Corinthians Brazil,Tony s pet project,Al Nassr Saudi Arabia,Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,Auntie back in fashion even if the dress look...,Shoaib appeal goes to the top,Hussain hurt by shambles but lays blame on e...,England s decade of disasters,Revenge is sweet for jubilant Cronje,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for,Christmas glitches,Upending a table Chopping a line and Scoring ...,Scientific evidence unreliable defence claims,Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake up of failing NHS,Lessons of law s hard heart
4,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers are you all whingers,Hollywood Beyond,Ashes and diamonds,Whingers a formidable minority,Alan Parker part two,Thuggery Toxins and Ties,...,Most everywhere UDIs,Most wanted Chloe lunettes,Return of the cane completely off the agenda,From Sleepy Hollow to Greeneland,Blunkett outlines vision for over s,Embattled Dobson attacks play now pay later ...,Doom and the Dome,What is the north south divide,Aitken released from jail,Gone aloft


In [43]:

#Converting headings into lowercase
for index in new_index:
  data[index] = data[index].str.lower()

data.shape


(3975, 25)

In [44]:
headlines = []
for row in range(0,len(data.index)):
  headlines.append(" ".join(str(x) for x in data.iloc[row,:]))
headlines[2]

'coventry caught on counter by flo united s rivals on the road to rio thatcher issues defence before trial by video police help smith lay down the law at everton tale of trautmann bears two more retellings england on the rack pakistan retaliate with call for video of walsh cullinan continues his cape monopoly mcgrath puts india out of their misery blair witch bandwagon rolls on pele turns up heat on ferguson party divided over kohl slush fund scandal manchester united  england  women in record south pole walk vasco da gama  brazil  south melbourne  australia  necaxa  mexico  real madrid  spain  raja casablanca  morocco  corinthians  brazil  tony s pet project al nassr  saudi arabia  ideal holmes show pinochet leaves hospital after tests useful links'

In [45]:
cv = CountVectorizer(ngram_range=(2,2))
train_data_cv = cv.fit_transform(headlines)

In [46]:
tv = TfidfVectorizer(ngram_range=(2,2))
train_data_tv = tv.fit_transform(headlines)


In [47]:
random_classifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
random_classifier.fit(train_data_cv, train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [48]:
random_classifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
random_classifier.fit(train_data_tv, train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [49]:
test_transform = []
for row in range(0, len(test.index)):
  test_transform.append(" ".join(str(x) for x in test.iloc[row, 2:27]))
test_data_cv = cv.transform(test_transform)
test_data_tv = tv.transform(test_transform)

In [50]:
matrix_cv = confusion_matrix(test['Label'], random_classifier.predict(test_data_cv))
score_cv = accuracy_score(test['Label'], random_classifier.predict(test_data_cv))
report_cv = classification_report(test['Label'], random_classifier.predict(test_data_cv))
matrix_tv = confusion_matrix(test['Label'], random_classifier.predict(test_data_tv))
score_tv = accuracy_score(test['Label'], random_classifier.predict(test_data_tv))
report_tv = classification_report(test['Label'], random_classifier.predict(test_data_tv))

In [53]:
print("Matrix_CV : \n",matrix_cv)
print()
print("Score_CV : \n",score_cv)
print()
print("Report_CV : \n",report_cv)
print()
print("Matrix_TV : \n",matrix_tv)
print()
print("Score_TV : \n",score_tv)
print()
print("Report_TV : \n",report_tv)

Matrix_CV : 
 [[138  48]
 [ 24 168]]

Score_CV : 
 0.8095238095238095

Report_CV : 
               precision    recall  f1-score   support

           0       0.85      0.74      0.79       186
           1       0.78      0.88      0.82       192

    accuracy                           0.81       378
   macro avg       0.81      0.81      0.81       378
weighted avg       0.81      0.81      0.81       378


Matrix_TV : 
 [[148  38]
 [ 17 175]]

Score_TV : 
 0.8544973544973545

Report_TV : 
               precision    recall  f1-score   support

           0       0.90      0.80      0.84       186
           1       0.82      0.91      0.86       192

    accuracy                           0.85       378
   macro avg       0.86      0.85      0.85       378
weighted avg       0.86      0.85      0.85       378

